In [48]:
import awkward as ak
import numpy as np
import pandas as pd
import json
import os
import shutil
import pathlib
from typing import List, Optional

from coffea import processor
from coffea.nanoevents.methods import candidate, vector
from coffea.analysis_tools import Weights, PackedSelection

import warnings
warnings.filterwarnings("ignore", message="Found duplicate branch ")

import pyarrow as pa
import pyarrow.parquet as pq
import pickle as pkl

In [49]:
# # from coffea casa
# import uproot
# f = uproot.open('root://xcache//store/mc/RunIISummer19UL17NanoAODv2/GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8/NANOAODSIM/106X_mc2017_realistic_v8-v1/250000/13D6BBD5-89E3-8647-AED6-FB5DFAAF4C8C.root:Events')
# f.num_entries   ### checks number of events

In [50]:
# import uproot
# sample = "GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8"
# for i in range(len(fileset[sample])):
#     f = uproot.open(fileset[sample][i])
#     num = f['Events'].num_entries   ### checks number of events per file        
#     print('number of events in file', i, 'is', num)

In [56]:
### load result of 2 jobs
sample = 'GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8'
year = '2017'
num_jobs = 2
sum_sumgenweight = 0

for i in range(num_jobs):
    tmp = pq.read_table(f'./outfiles/{i}-{i+1}_ele.parquet')
    tmp = tmp.to_pandas()
    if i==0:
        data = tmp
    else:
        data = pd.concat([data,tmp], ignore_index=True)
        
    # load and sum the sumgenweight of each
    with open(f'./outfiles/{i}-{i+1}.pkl', 'rb') as f:
        metadata = pkl.load(f)
    sum_sumgenweight = sum_sumgenweight + metadata[sample][year]['sumgenweight']

In [57]:
print(len(data))
data.head()

645


,lepton_pt,lep_isolation,met,ht,mt_lep_met,dr_jet_candlep,weight
0,78.507706,0.013055,33.577034,499.12500,52.215054,0.471214,28.893
1,101.503654,0.005712,24.996897,420.18750,11.229020,0.388044,28.893
2,80.132812,0.011502,53.236359,436.25000,30.718420,0.238419,28.893
3,224.800964,0.014546,20.161127,598.84375,18.302195,0.029374,28.893
4,91.291237,0.000000,31.520144,484.12500,5.314937,0.359515,28.893
